In [ ]:
plt.figure()
for i in range(len(jaw_instance)):
    data=jaw_instance.getTeethByIdx(i).data
    data_smashed=data[:,:2]
    west=data_smashed[np.argmin(data_smashed,axis=0)][0]
    east=data_smashed[np.argmax(data_smashed,axis=0)][0]
    north=data_smashed[np.argmax(data_smashed,axis=0)][-1]
    south=data_smashed[np.argmin(data_smashed,axis=0)][-1]
    d1=(east[1]-south[1])/(east[0]-south[0])
    b1=east[1]-east[0]*d1
    d2=(north[1]-east[1])/(north[0]-east[0])
    b2=north[1]-north[0]*d2

    plt.scatter(data_smashed[:,0],data_smashed[:,1])
    x=int(np.min(data[:,0]))
    y=int(np.max(data[:,0]))
    #plt.plot(range(x,y),line(d1,b1,[x,y]))
    plt.plot(range(x,y),line(d2,b2,[x,y])-10)
    plt.axis("equal")
plt.show()

In [ ]:
from scipy.spatial import ConvexHull

In [ ]:
plt.figure()
data_smashed=list()
for i in range(len(jaw_instance)):
    data=jaw_instance.getTeethByIdx(i).data
    data_smashed.append(np.mean(data[:,:2],axis=0))
data_smashed=np.array(data_smashed)
z = np.polyfit(data_smashed[:,0], data_smashed[:,1], 2)
f = np.poly1d(z)
x_new = np.linspace(-30, 30, 50)
y_new = f(x_new)
plt.scatter(data_smashed[:,0],data_smashed[:,1])
dict_tooth_obj=dict()
for i,point in enumerate(data_smashed):
    a=tangent(point[0],z)
    if np.abs(a)<1e-1:
        b=point[1]-a*point[0]
        plt.plot(range(int(point[0])-3,int(point[0])+3),line(a,b,[int(point[0])-3,int(point[0])+3]),c='r')
        plt.plot(range(int(point[0])-2,int(point[0])),line(-1/a,-b/2,[int(point[0])-2,int(point[0])])/2,c='y')
    else:
        b=point[1]-a*point[0]
        plt.plot(range(int(point[0])-3,int(point[0])+3),line(a,b,[int(point[0])-3,int(point[0])+3]),c='r')
        plt.plot(range(int(point[0])-3,int(point[0])+3),line(-1/a,-b/2,[int(point[0])-3,int(point[0])+3]),c='y')
    dict_tooth_obj[i]=[-1/a]
plt.plot( x_new, y_new)
plt.axis("equal")
plt.show()

In [ ]:
plt.scatter(data_smashed[:,0],data_smashed[:,1])
for i in range(len(jaw_instance)):
    data=jaw_instance.getTeethByIdx(i).data
    data_smashed_=data[:,:2]
    east=data_smashed_[np.argmax(data_smashed_,axis=0)][0]
    north=data_smashed_[np.argmax(data_smashed_,axis=0)][-1]
    d2=(north[1]-east[1])/(north[0]-east[0])
    b2=north[1]-north[0]*d2
    point=np.mean(data_smashed_,axis=0)
    a=tangent(point[0],z)
    if np.abs(a)<1e-1:
        b=point[1]-a*point[0]
        plt.plot(range(int(point[0])-3,int(point[0])+3),line(d2,b2,[int(point[0])-3,int(point[0])+3]),c='r')
        plt.plot(range(int(point[0])-2,int(point[0])),line(-1/a,-b/2,[int(point[0])-2,int(point[0])]),c='y')
    else:
        b=point[1]-a*point[0]
        plt.plot(range(int(point[0])-3,int(point[0])+3),line(d2,b2,[int(point[0])-3,int(point[0])+3]),c='r')
        plt.plot(range(int(point[0])-3,int(point[0])+3),line(-1/a,-b/2,[int(point[0])-3,int(point[0])+3]),c='y')

plt.plot( x_new, y_new)
plt.axis("equal")
plt.show()

In [ ]:

from itertools import combinations
def func(l_):
    pairs = list(combinations(l_, 2))
    coef=list()
    for i,elem in enumerate(pairs):
        a=elem[0][1]-elem[1][1]
        b=elem[0][0]-elem[1][0]
        coef.append(a/b)
    pairscoef=list(combinations(coef,2))
    angle=list()
    for j,elem in enumerate(pairscoef):
        a=getAngle(elem[0],elem[1])
        angle.append(a)
    key=pairscoef[np.argmax(np.array(angle))]
    return key

dict_tooth=dict()
for i in range(len(jaw_instance)):
    print(i)
    plt.figure()
    data=jaw_instance.getTeethByIdx(i).data
    data_smashed=data[:,:2]
    plt.scatter(data_smashed[:,0],data_smashed[:,1])
    hull = ConvexHull(data_smashed)
    l_=list(data_smashed[hull.vertices])
    keys=func(l_)
    plt.scatter(data_smashed[hull.vertices][:,0],data_smashed[hull.vertices][:,1])
    plt.plot(data_smashed[hull.vertices,0], data_smashed[hull.vertices,1], 'r--', lw=2)
    for key in keys:
        plt.plot(range(int(point[0])-3,int(point[0])+3),line(key,0,[int(point[0])-3,int(point[0])+3]),c='b')
    dict_tooth[i]=keys

rotation=dict()
for i,key in enumerate(dict_tooth.keys()):
    objective=dict_tooth_obj[key][0]
    potential=dict_tooth[key]
    list_angle=[getAngle(objective,potential[0]),getAngle(objective,potential[1])]
    if i==13:
        print(objective,potential)
    rotation[key]=np.min(list_angle)


for i in range(len(jaw_instance)):
    teeth=jaw_instance.getTeethByIdx(i)
    teeth.rotate_teeth(rotation[i],2)

In [ ]:
def getAngle(a1,a2):
    sup=a2-a1
    bottom=1+a1*a2
    if bottom>1e-4:
        return np.arctan(np.abs(sup/bottom))
    return np.pi/2

def tangent(x,z):
    return z[0]*2*x+z[1]

def line(a,b,range_):
    res=[]
    for x in range(range_[0],range_[1]):
        res.append(a*x+b)
    return np.array(res)

In [ ]:
from scipy.spatial import ConvexHull

def distance(p1,p2,p3):
    return np.linalg.norm(np.cross(p2-p1, p1-p3))/np.linalg.norm(p2-p1)
def distance2(a,p):
    b=(1,-a)
    return np.linalg.norm(np.cross(b,p))/np.linalg.norm(b)
def func(l_,list_lines):
    elem=l_[0]
    for key in list_lines.keys():
        if distance2(key[0],elem)<1e-1:
            list_lines[key].append(elem)
            l_.pop(0)
            continue
    try:
        t=True
        elem2=l_[1]
        elem3=l_[2]
        if distance(elem,elem2,elem3)<1e1:
            a=(elem[1]-elem2[1])/(elem[0]-elem2[0])
            b=elem[1]-elem[0]*a
            for key in list_lines.keys():
                if np.abs(a-key[0])<1.5:
                    list_lines[key].append([elem,elem2,elem3])
                    l_.pop(0)
                    l_.pop(1)
                    l_.pop(2)
                    t=False
            if t:
                list_lines[(a,b,tuple(elem))]=[elem,elem2,elem3]
                l_.pop(0)
                l_.pop(1)
                l_.pop(2)
    except IndexError:
        None
    np.random.shuffle(l_)
    return l_,list_lines